In [1]:
from PIL import Image
from qiskit import *
import time
import numpy as np

In [2]:
from quantograph import *
import runes

In [3]:
def show_image(image):
    img = Image.new('RGB',(8,8))
    for x in range(img.size[0]):
        for y in range(img.size[1]):
            img.load()[x,y] = image[x,y]
    img = img.resize((256,256))
    img.show()

In [4]:
n = 6
backend = Aer.get_backend('statevector_simulator')
left = QuantumRegister(n)
right = QuantumRegister(n)

In [5]:
def renderer(left_image,right_image,frame_num,method='A'):
    
    left_state = image2state( left_image, grid )
    right_state = image2state( right_image, grid )

    filenames = {'left':[],'right':[]}
    
    for f in range(frame_num):
        
        print( (f+1)*100/frame_num )

        circuits = []
        for j in range(3):
            qc = QuantumCircuit(left,right)
            qc.initialize(left_state[j],left)
            qc.initialize(right_state[j],right)
            
            if method=='A':
                qc.cx(left,right)
                qc.rx( np.pi*f/(2*(frame_num-1)), left )
                qc.cz(right,left)
                qc.rx( -np.pi*f/(2*(frame_num-1)), left )
                qc.cz(right,left)
                qc.cx(left,right)
            elif method=='B':
                if (f<int(frame_num/3)):
                    frame_num1 = int(frame_num/3)
                    qc.rx( np.pi*f/(2*(frame_num1-1)), right )
                    qc.cz(left,right)
                    qc.rx( -np.pi*f/(2*(frame_num1-1)), right )
                    qc.cz(left,right)
                    print( (f+1)/frame_num1 )
                elif (f<int(2*frame_num/3)):
                    frame_num1 = int(frame_num/3)
                    frame_num2 = int(2*frame_num/3)-frame_num1
                    ff = f-frame_num1
                    qc.cx(left,right)
                    qc.rx( np.pi*(ff+1)/(2*frame_num2), left )
                    qc.cz(right,left)
                    qc.rx( -np.pi*(ff+1)/(2*frame_num2), left )
                    qc.cz(right,left)
                    print( (ff+1)/frame_num2 )
                else:
                    frame_num1 = int(frame_num/3)
                    frame_num2 = int(2*frame_num/3)-frame_num1
                    frame_num3 = frame_num - int(2*frame_num/3)
                    ff = f-frame_num2-frame_num1
                    qc.cx(left,right)
                    qc.cx(right,left)
                    qc.rx( np.pi*(ff+1)/(2*frame_num3), right )
                    qc.cz(left,right)
                    qc.rx( -np.pi*(ff+1)/(2*frame_num3), right )
                    qc.cz(left,right)
                    print( (ff+1)/frame_num3 )
            else:    
                qc.rx( np.pi*f/(2*(frame_num-1)), right )
                qc.cz(left,right)
                qc.rx( -np.pi*f/(2*(frame_num-1)), right )
                qc.cz(left,right)
                qc.rx( np.pi*f/(2*(frame_num-1)), left )
                qc.cz(right,left)
                qc.rx( -np.pi*f/(2*(frame_num-1)), left )
                qc.cz(right,left)
                qc.rx( np.pi*f/(2*(frame_num-1)), right )
                qc.cz(left,right)
                qc.rx( -np.pi*f/(2*(frame_num-1)), right )
                qc.cz(left,right)
                
            circuits.append( qc )

        job = execute(circuits, backend, shots=1)

        counts = {'left':[],'right':[]}
        for j in range(3):
            full_counts = ket2counts( job.result().get_statevector(circuits[j]) )

            left_counts = {}
            right_counts = {}
            for full_string in full_counts:
                left_string = full_string[6:12]
                right_string = full_string[:6]
                try:
                    left_counts[left_string] += full_counts[full_string]
                except:
                    left_counts[left_string] = full_counts[full_string]
                try:
                    right_counts[right_string] += full_counts[full_string]
                except:
                    right_counts[right_string] = full_counts[full_string]

            counts['left'].append( left_counts )
            counts['right'].append( right_counts )

        left_frame = counts2image(counts['left'],grid)
        right_frame = counts2image(counts['right'],grid)

        for side in ['left','right']:
            filename = 'outputs/temp_'+side+'_'+str(f)+'.png'
            if side=='left':
                frame = left_frame
            else:
                frame = right_frame
            save_image( frame, scale=[300,300], filename=filename)
            filenames[side].append( filename )

    animation = {'left':'','right':''}        
    for side in ['left','right']:
        t = time.localtime()
        animation[side] = 'outputs/'+side+'_'+str(t.tm_year)+'_'+str(t.tm_mon)+'_'+str(t.tm_mday)+'@'+str(t.tm_hour)+':'+str(t.tm_min)+':'+str(t.tm_sec)+'.png'
        APNG.from_files(filenames[side],delay=250).save( animation[side] )

    for side in ['left','right']:
        for filename in filenames[side]:
            os.remove(filename)

    return animation

In [12]:
frame_num = 20
animation = renderer(runes.rune6,runes.rune7,frame_num,method='A')
display(Markdown('![]('+animation['left']+')'))
display(Markdown('![]('+animation['right']+')'))

5.0
10.0
15.0
20.0
25.0
30.0
35.0
40.0
45.0
50.0
55.0
60.0
65.0
70.0
75.0
80.0
85.0
90.0
95.0
100.0


![](outputs/left_2019_11_3@20:13:34.png)

![](outputs/right_2019_11_3@20:13:34.png)